In [36]:
import os
import math

import numpy as np
import fasttext.util

import utils


embeddings = utils.get_embeddings(["en", "fr"])

en_emb = embeddings["en"]
fr_emb = embeddings["fr"]
print("yes")

yes


In [37]:
class TranslationDataset():
    def __init__(self, inputs, targets, embeddings):
        self.inputs = inputs 
        self.targets = targets 
        
        # Encoders for both languages
        en_i = embeddings["en"]
        en_t = embeddings["fr"]
        
        self.input_em = []
        for seq in inputs:
            self.input_em.append(np.array([en_i.get_word_vector(w) for w in seq.split()]))
            
        self.target_em = []
        for seq in targets:
            self.target_em.append(np.array([en_t.get_word_vector(w) for w in seq]))

    def __len__(self):
        return (len(self.sequence))

    def __getitem__(self, idx):
        return({
            "input":self.inputs[idx],
            "target":self.targets[idx],
            "input_embedding":self.input_em[idx],
            "target_embedding":self.target_em[idx],
        })
    
data = ["the cat likes oranges", "hello my friend"]
target = ["le chat aime les oranges", "bonjour mon amie"]

dataset = TranslationDataset(data, target, embeddings) 

print(dataset[0])

{'input': 'the cat likes oranges', 'target': 'le chat aime les oranges', 'input_embedding': array([[ 1.66990623e-01, -1.18591875e-01,  2.26854086e-02,
         9.37557518e-02,  4.71362472e-02,  3.38186175e-02,
         7.64891580e-02, -8.32068846e-02,  2.73817405e-02,
         3.21494825e-02,  4.83908653e-02,  1.49061069e-01,
         2.08143353e-01, -2.12923363e-02,  9.45081711e-02,
         1.43020265e-02, -3.42536308e-02,  7.64139444e-02,
        -2.43578311e-02,  9.10279453e-02,  1.07339360e-02,
         9.78427008e-02, -6.92236237e-03, -1.38092190e-01,
        -6.40805252e-03, -6.01041615e-02, -9.97035354e-02,
         6.42363541e-03, -6.74770214e-03, -1.28166586e-01,
         8.96790773e-02,  1.56000666e-02,  7.70670623e-02,
         2.00899839e-02,  2.49394104e-02, -4.41633128e-02,
        -6.98457882e-02,  3.94032858e-02,  3.63518372e-02,
         1.46088660e-01,  1.01135530e-01,  3.57998013e-02,
         2.66445614e-02, -4.17234525e-02, -1.47688538e-02,
         4.26694490e-02

In [67]:
def pos_encoding(seq):
    seq_len = seq.shape[0]
    d_model = seq.shape[1]
    
    encoding = []
    for i in range(seq_len):
        w = 1 / (10000 ** ((2 * i) / d_model))

        wi_s = [math.sin(p * w) * (i % 2) for p in range(d_model)]
        wi_c = [math.cos(p * w) * ((i + 1) % 2) for p in range(d_model)]
        
        encoding.append(np.add(wi_s, wi_c))
        
    encoding = np.array(encoding)
    
    return np.add(encoding, seq)
    

print(dataset[0]["input"])
pos_encoding(dataset[0]["input_embedding"])

the cat likes oranges


array([[ 1.16699062,  0.42171043, -0.39346143, -0.89623674, -0.60650737,
         0.3174808 ,  1.03665944,  0.67069537, -0.11811829, -0.87898078,
        -0.79068066,  0.15348677,  1.05199731,  0.88615445,  0.23124539,
        -0.74538589, -0.99191311, -0.19874939,  0.63595888,  1.07973256,
         0.418816  , -0.44988656, -1.00688319, -0.67092521,  0.41777095,
         0.93109865,  0.54721579, -0.28571517, -0.96935357, -0.87622412,
         0.24393053,  0.93034242,  0.91129042,  0.00681324, -0.82363086,
        -0.94785552, -0.19780948,  0.80481734,  0.99142548,  0.41273159,
        -0.56580253, -0.95153948, -0.37334075,  0.51338985,  0.98507445,
         0.56799144, -0.16484521, -0.8782033 , -0.94007649,  0.11915812],
       [ 0.01792766,  0.46820304,  0.73011209,  0.51322528,  0.68460212,
        -0.62119828, -0.63443907, -0.72481344, -0.93191297,  0.09682753,
         0.48711193,  0.99541023,  0.73732295,  0.49834952, -0.47047961,
        -1.10980251, -1.0353997 , -0.76442392,  0.